In [28]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"

In [29]:
from datasets import load_dataset
datasets = load_dataset("squad")

In [30]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [31]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [33]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [34]:
tokenizer("Hello World")

{'input_ids': [101, 7592, 2088, 102], 'attention_mask': [1, 1, 1, 1]}

In [35]:
max_length = 384 
doc_stride = 128 
# 251,252 是两条条超过max_length的记录
test_251 = datasets['train'][251]
test_252 = datasets['train'][252]
test_example = {
    'id': [test_251['id'], test_252['id']],
    'title': [test_251['title'], test_252['title']],
    'context': [test_251['context'], test_252['context']],
    'question': [test_251['question'], test_252['question']],
    'answers': [test_251['answers'], test_252['answers']],
}

In [36]:
test_example

{'id': ['5733caf74776f4190066124e', '5733caf74776f4190066124f'],
 'title': ['University_of_Notre_Dame', 'University_of_Notre_Dame'],
 'context': ["The men's basketball team has over 1,600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 NCAA tournaments. Former player Austin Carr holds the record for most points scored in a single game of the tournament with 61. Although the team has never won the NCAA Tournament, they were named by the Helms Athletic Foundation as national champions twice. The team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending UCLA's record 88-game winning streak in 1974. The team has beaten an additional eight number-one teams, and those nine wins rank second, to UCLA's 10, all-time in wins against the top team. The team plays in newly renovated Purcell Pavilion (within the Edmund P. Joyce Center), which reopened for the beginning of the 2009–2010 season. The team is coached by Mik

In [37]:
def prepare_train_features(examples):
    """
    整体方法是在标准的tokenizer基础上，增加了start_positions和end_positions两列，代表答案token的开始和结束位置
    """
    if isinstance(examples["question"], str):
        examples["question"] = examples["question"].lstrip()
    else:
        examples["question"] = [q.lstrip() for q in examples["question"]]
        
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 切片后的chunk => 原文
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # token pos => char pos
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i] # chunk token

        cls_index = input_ids.index(tokenizer.cls_token_id) # cls pos, always 0

        sequence_ids = tokenized_examples.sequence_ids(i) # token mask，0-question，1-answer，None-special token
        
        sample_index = sample_mapping[i] # 对应的原文
        
        answers = examples["answers"][sample_index] # 对应原文的答案

        if len(answers["answer_start"]) == 0: # 'answers': {'text': [''], 'answer_start': []}}，原文中没有答案
            # 开始结束位置记录为 0
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 答案在原文中的开始和结束位置
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0 # token对应的context开始的位置
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1 

            token_end_index = len(input_ids) - 1 # token对应的context结束的位置
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1 
            
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                # 当前chunk中没有答案（必须完整包含）
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 当前chunk中有答案
                # 后移token的位置，找到答案对应的token开始位置
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                # 前移token的位置，找到答案对应的token结束位置
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [38]:
test_tokenized_example = prepare_train_features(test_example)

In [39]:
tokenized_datasets = datasets.map(prepare_train_features,
                                  batched=True,
                                  remove_columns=datasets["train"].column_names)

In [40]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 88524
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10784
    })
})

In [23]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
batch_size=192
model_dir = f"models/{model_checkpoint}-finetuned-squad"

args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [42]:
from transformers import default_data_collator

data_collator = default_data_collator

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [44]:
trainer.train()
trainer.save_model(model_dir)
trainer.save_state()

/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.428168
2,2.157600,1.275152
3,1.303900,1.251565


In [60]:
from tqdm.auto import tqdm
import collections
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    """
    根据特征和预测还原到原本的答案
    """
    all_start_logits, all_end_logits = raw_predictions
    # 构建一个从示例到其对应特征的映射。
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # 我们需要填充的字典。
    predictions = collections.OrderedDict()

    # 日志记录。
    print(f"正在后处理 {len(examples)} 个示例的预测，这些预测分散在 {len(features)} 个特征中。")

    # 遍历所有示例！
    for example_index, example in enumerate(tqdm(examples)):
        # 这些是与当前示例关联的特征的索引。
        feature_indices = features_per_example[example_index]

        min_null_score = None # 仅在squad_v2为True时使用。
        valid_answers = []
        
        context = example["context"]
        # 遍历与当前示例关联的所有特征。
        for feature_index in feature_indices:
            # 我们获取模型对这个特征的预测。
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 这将允许我们将logits中的某些位置映射到原始上下文中的文本跨度。
            offset_mapping = features[feature_index]["offset_mapping"]

            # 更新最小空预测。
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # 浏览所有的最佳开始和结束logits，为 `n_best_size` 个最佳选择。
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # 不考虑长度小于0或大于max_answer_length的答案。
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # 在极少数情况下我们没有一个非空预测，我们创建一个假预测以避免失败。
            best_answer = {"text": "", "score": 0.0}
        
        # 选择我们的最终答案：最佳答案或空答案（仅适用于squad_v2）
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [46]:
def prepare_validation_features(examples):
    """
    将验证数据处理为features
    """
    # 一些问题的左侧有很多空白，这些空白并不有用且会导致上下文截断失败（分词后的问题会占用很多空间）。
    # 因此我们移除这些左侧空白
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和可能的填充对我们的示例进行分词，但使用步长保留溢出的令牌。这导致一个长上下文的示例可能产生
    # 几个特征，每个特征的上下文都会稍微与前一个特征的上下文重叠。
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例在上下文很长时可能会产生几个特征，我们需要一个从特征映射到其对应示例的映射。这个键就是为了这个目的。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 我们保留产生这个特征的示例ID，并且会存储偏移映射。
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # 获取与该示例对应的序列（以了解哪些是上下文，哪些是问题）。
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # 一个示例可以产生几个文本段，这里是包含该文本段的示例的索引。
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将不属于上下文的偏移映射设置为None，以便容易确定一个令牌位置是否属于上下文。
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [47]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [48]:
validation_features

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10784
})

In [49]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
trained_trainer = Trainer(
    trained_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# 基于featture的原始预测结果
raw_predictions = trained_trainer.predict(validation_features)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [56]:
print(raw_predictions)
print(len(raw_predictions.predictions[0]))
print(len(raw_predictions.predictions[0][0]))

PredictionOutput(predictions=(array([[-5.286222 , -6.612807 , -7.293781 , ..., -7.5817404, -7.56017  ,
        -7.627872 ],
       [-5.298247 , -6.6009326, -7.273103 , ..., -7.583229 , -7.5616307,
        -7.628161 ],
       [-2.720223 , -5.06894  , -6.461087 , ..., -7.641131 , -7.598675 ,
        -7.589619 ],
       ...,
       [-2.084156 , -6.447255 , -6.801171 , ..., -7.570975 , -7.555427 ,
        -7.5553794],
       [-1.9123228, -6.6626377, -6.757734 , ..., -7.5725675, -7.5949917,
        -7.569608 ],
       [-1.7271022, -6.8095   , -6.809823 , ..., -7.6145835, -7.5947175,
        -7.5965524]], dtype=float32), array([[-4.779095 , -7.073431 , -6.561958 , ..., -7.4892554, -7.509979 ,
        -7.4518743],
       [-4.8729267, -7.0950127, -6.587563 , ..., -7.4872966, -7.5079274,
        -7.4510503],
       [-2.4264112, -6.2001505, -7.2404637, ..., -7.432796 , -7.484364 ,
        -7.4749546],
       ...,
       [-2.0083191, -7.132052 , -7.415781 , ..., -7.5316133, -7.5426645,
        -7

In [63]:
from datasets import load_metric

# 基于原数据、特征、预测结果还原回答
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

# 标准答案
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
# 计算准确率和f1 score
metric = load_metric("squad")
metric.compute(predictions=formatted_predictions, references=references)

正在后处理 10570 个示例的预测，这些预测分散在 10784 个特征中。


  0%|          | 0/10570 [00:00<?, ?it/s]

/tmp/ipykernel_900124/2594762423.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'exact_match': 71.57048249763481, 'f1': 81.15865261806042}